# 1) ETL (b) Extraction

In [2]:
from settings import GEMINI_API_KEY
from data.ingest import GoogleDriveExtractor

extractor = GoogleDriveExtractor(
    credentials_file="smart-platform.json", 
    sheet_name="Master Database",
    worksheet_name="inventory", 
    gemini_api_key=GEMINI_API_KEY 
)

In [3]:
folder_details_dict = extractor.view_folder_details(folder_link="https://drive.google.com/drive/folders/103epRgaKBSrXuLI7dhnfVLPs4d3bj94o?usp=drive_link",
                              credentials_file = "smart-platform.json")

Folder Name: 01 Malacca (Shaz), Share Link: https://drive.google.com/drive/folders/1nstotWI9LYvUamNw-NSew-jnVDD7VAaH, Item Count: 2
Folder Name: 17 Others, Share Link: https://drive.google.com/drive/folders/1pRG5W1oJEc8Ify3XQ0iWY1NXM-63FzhN, Item Count: 7
Folder Name: 16 Umrah (Norra & Zul), Share Link: https://drive.google.com/drive/folders/1AQW3PViswZN18QRPTAhAdgrt9fxi6xsq, Item Count: 16
Folder Name: 15 Perak (Malena), Share Link: https://drive.google.com/drive/folders/1Ne8NVDgbbvDvn4OKnzW3bQDl6g-y6Llx, Item Count: 3
Folder Name: 14 Perlis (Noey), Share Link: https://drive.google.com/drive/folders/1GlEZiVla7ZQk1u8EcOVCM84nu_hXDwVk, Item Count: 9
Folder Name: 13 Kedah (Noey), Share Link: https://drive.google.com/drive/folders/1eEu1iI23EAVTAjlzFGM3sF5XY1NIAAZF, Item Count: 18
Folder Name: 12 Kelantan (Noey), Share Link: https://drive.google.com/drive/folders/1ohGD8AgMID2IdT3oo3bU_QZQatM6eC0x, Item Count: 1
Folder Name: 11 Sabah (Zul), Share Link: https://drive.google.com/drive/folders

In [6]:
list(folder_details_dict.keys())[3:7]

['15 Perak (Malena)',
 '14 Perlis (Noey)',
 '13 Kedah (Noey)',
 '12 Kelantan (Noey)']

In [7]:
# restarted from sabah
# folder_link = folder_details_dict['10 Sarawak (Norra)']+'?usp=drive_link'
for folder_name in list(folder_details_dict.keys())[3:7]:
    print(f"=====> {folder_name}")
    folder_link = folder_details_dict[folder_name]+'?usp=drive_link'
    print(folder_link)
    extractor.run_ETL(folder_link)


=====> 15 Perak (Malena)
https://drive.google.com/drive/folders/1Ne8NVDgbbvDvn4OKnzW3bQDl6g-y6Llx?usp=drive_link
Found 3 PDF files in the specified folder.
1 ---> Processing: partners - perak - accommodation/tingkat valley 
Downloading: partners - perak - accommodation/tingkat valley 
Download 100%
Downloaded: partners - perak - accommodation/tingkat valley 
Performing OCR on: partners - perak - accommodation/tingkat valley 
file_path: partners - perak - accommodation/tingkat valley.pdf --> image_path: partners - perak - accommodation/tingkat valley_page_1.jpg
file_path: partners - perak - accommodation/tingkat valley.pdf --> image_path: partners - perak - accommodation/tingkat valley_page_2.jpg
Updating Google Sheet with: partners - perak - accommodation/tingkat valley 
Data appended to 'Master Database' - 'inventory'
2 ---> Processing: partners - perak - nature/tasik cermin ipoh sdn bhd
Downloading: partners - perak - nature/tasik cermin ipoh sdn bhd
Download 100%
Downloaded: partner

# 1) ETL (c) Transformation

In [8]:
from gdrive.gdrive_handler import GspreadHandler

worksheet_name = "inventory_processed"
worksheet_name = "inventory"
def get_df(sheet_name, worksheet_name):
    gspread_handler = GspreadHandler(credentials_filepath='smart-platform.json')
    df = gspread_handler.get_sheet_as_df(sheet_name=sheet_name, worksheet_name=worksheet_name)
    return df

df_raw = get_df(sheet_name = "Master Database" , worksheet_name="inventory")
df = get_df(sheet_name = "Master Database" , worksheet_name="inventory_processed")
# drop duplicates in Title column



In [9]:
df_raw.drop_duplicates(subset=['Title'], keep='last')

,Destination,Title,Text
1,partners - malacca - attractions,menara taming sari,MENARA TAMING SARI\nMELAKA\n\nTaming Sari Tow...
2,partners - malacca - attractions,melaka river cruise,MELAKA RIVER CRUISE\nS.M.A. MELAKA\n\nwww.mel...
35,partners - kelantan - culture,rs travel sdn bhd,SPECIAL OFFER\nRIS Travel\nSdn Bhd\nRIS TRAVE...
41,partners - sabah - adventure water sports,riverbug asia the white water,[Image of a flyer for a white water rafting t...
42,partners - sabah -accommodation,ara dinawan island resort,Ara Dinawan Island\n\nAra Dinawan Island Reso...
...,...,...,...
111,partner - selangor - water park,wet world shah alam,VOTED NO.1\nNEIGHBOURHOOD\nWATER PARK\n\nOrig...
112,partner - selangor - land park,play gamuda land,"# Pay less, play more!\n## Discovery Park\n##..."
113,partner - selangor - nature life,fraser valley kuala kubu bharu,# CHOICES OF ACCOMODATION \n## 6 PAX OF TENT\...
114,partner - selangor - tours,dasia travel,**Price Per Couple**\n\n3D2N Lombok Honeymoon...


In [10]:
# show rows where sabah is in Destination column
view_df = df_raw[df_raw['Destination'].str.contains('kelantan', case=False)]
print(f"len: {len(view_df)}")
view_df

len: 1


,Destination,Title,Text
35,partners - kelantan - culture,rs travel sdn bhd,SPECIAL OFFER\nRIS Travel\nSdn Bhd\nRIS TRAVE...


In [17]:
df

,Item,Country,Location,Type,Tags,Duration,Title,Description,The data ingested isn't shown on the flyers.,Vendor ID,Website Ingestion instead?,Complexity Level & Error Rate,Completion Rate & Accurate Text Capture Rate,Ranking
0,1,Malaysia,Putrajaya,Activity,"Tours, Sightseeing, Landmarks, Monument, Boat ...","25 minutes, 30 minutes, 45 minutes",Cruise Tasik Putrajaya,**Activities Description:**\nCruise Tasik Putr...,this process started from item #6 onwards,WPP01,www.cruisetasikputrajaya.com,10% \n\n- route: incomplete & inaccurate\n- ti...,90%,
1,2,Malaysia,Malacca,Activity,"Attractions, Tours, Sightseeing, Landmarks",7 minutes,Menara Taming Sari,**Activities Description:**\n* A 360° viewing ...,,MLK01,https://www.menaratamingsari.com/,"30% \n\n- activity description: incomplete, to...",70%,
2,3,Malaysia,Malacca,Activity,"Tours, Sightseeing, Landmarks, Monument, Boat ...",45 minutes,Melaka River Cruise,**Activities Description:**\n- **Melaka River ...,,MLK02,www.melakarivercruise.my,40% \n\n- timing: inaccurate and not properly...,60%,
3,4,Malaysia,Terengganu,Accommodation,"Mid-Range Hotels (3-Star), Beachfront Hotels",3D2N,Perhentian Island Resort,**Pricings:**\n**Perhentian Discovery Package:...,,TRG01,www.perhentianislandsresort.net,60% \n\nconclusion: data ingested was summariz...,40%,
4,5,Malaysia,Terengganu,Activity,"Nature and Outdoors, Camping, Waterfalls, Natu...","Day trip, 2D1N, 3D2N",AG Holiday Sdn Bhd,**Pricings:**\n- KEM PELAJAR & IPT\n**Day Trip...,,TRG02,,90% \n\nconclusion: data ingested was summariz...,10%,
5,6,Malaysia,terengganu,"activity, accomodation",,,tenggol coral beach resort,**Tenggol Coral Beach Resort Snorkeling Packag...,,,,,,
6,7,Malaysia,johor,"activity, accomodation",,,legoland,**Activities Description**\n\n* **Annual Pass:...,,,,,,
7,8,Malaysia,negeri sembilan,activity,,,sky ladder pineapple farm,**Activities Description:**\n- Farm tram ride\...,,,,,,
8,9,Malaysia,negeri sembilan,"activity, accomodation",,,dataran segar city,**Activity Descriptions:**\n\n**Segar City 1-D...,,,,,,
9,10,Malaysia,negeri sembilan,accomodation,,,tasik villa international resort port dickson,**Activities:**\n- NA\n\n**Highlights:**\n- Fa...,,,,,,


In [7]:
# select only rows with 'Destination' column containing 'pahang'
df1 = df[df['Destination'].str.contains('sarawak', case=False)]
df1

,Destination,Title,Text


In [ ]:
# for destination replace all with rows with just the destination so select the word between - and -
# select the word after the last - for Destination column
df["Destination"] = df['Destination'].str.extract(r'-(.*)$')
df["Type"] = df['Destination'].str.extract(r'-(.*)$')
# now Destination column should only have the destination name
df["Destination"] = df['Destination'].str.extract(r'(.*)-')

In [ ]:
print(df["Text"].iloc[0])

In [ ]:
import pandas as pd
from llm_handler.GHandler import GHandler

# Country	Location	Type	Tags	Duration	Title	Description

def generate_activity_description(text_i):
    ghandler = GHandler(GEMINI_API_KEY, generation_config={"temperature": 0.95, "top_p": 0.95, "top_k": 40, "max_output_tokens": 40000}, block_threshold="BLOCK_NONE")
    prompt = f"""You are a travel agent that is given pure text from a travel brochure. USING ONLY THE TEXT given, you are to
    Summarize the text into activities description (include accurate comprehensive highlights), timings (if available), pricings (if available), contact informations (if available) and containing terms and conditions (if available).
    If there are available information for each of the content required, you can leave those fields empty with NA.
    text: {text_i}"""
    response = ghandler.prompt(prompt=prompt, model_name="gemini-pro")
    return response.text

# Apply the LLM function efficiently
df["Description"] = df["Text"].apply(generate_activity_description)  # Apply to all rows 

# (Optional) Handle potential errors from the LLM
df["Description"] = df["Description"].fillna("Error occurred") 

# Example Printing for First 3 rows
for description in df["Description"].iloc[0:3]:
    print(description)


In [ ]:
df

In [ ]:
from gdrive.gdrive_handler import GspreadHandler
CREDENTIALS_FILE = 'smart-platform.json'
SHEET_NAME = "Master Database" 
WORKSHEET_NAME = "inventory_processed"

gspread_handler = GspreadHandler(credentials_filepath=CREDENTIALS_FILE)
def update_google_sheet(df):
    """Updates the Google Sheet with the extracted text."""
    # keep only the columns we want to update
    df = df[["Destination", "Title", "Type", "Description"]]
    gspread_handler.update_cols(df, SHEET_NAME, WORKSHEET_NAME) # replace with the correct sheet name 

update_google_sheet(df)